In [6]:
import os
import torch
from models.wta_cnp import WTA_CNP


root_path = "/home/yigit/projects/mbcnp/outputs/experimental/62D/"
run_id, model_folder = f'1700927469', 'saved_models/'
run_folder = f'{root_path}{run_id}/'
models_path = f'{run_folder}{model_folder}'

wta_model_path = f'{models_path}wta_on_synth.pt'

y = torch.load(f'{run_folder}y.pt').cpu()
num_samples, t_steps, dy = y.shape
dx = 1
batch_size = 1

x = torch.linspace(0, 1, t_steps).repeat(batch_size, 1).view(batch_size, -1, 1)
colors = ['tomato', 'aqua', 'limegreen', 'gold', 'mediumslateblue', 'lightcoral', 'darkorange', 'teal']

wta = WTA_CNP(1, dy, 10, 10, [512, 512, 512], num_decoders=1, decoder_hidden_dims=[512, 512, 512], batch_size=batch_size)

wta.load_state_dict(torch.load(wta_model_path))
wta.eval()

WTA_CNP(
  (encoder): Sequential(
    (0): Linear(in_features=63, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
  (decoders): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=513, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=124, bias=True)
    )
  )
  (gate): Sequential(
    (0): Linear(in_features=512, out_features=1, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [7]:
def get_batch(idx=[0], cond_idx=[0]):
    batch_obs = torch.zeros(batch_size, len(cond_idx), dy+dx, dtype=torch.float32)
    batch_obs[:, :, 0] = x[0, cond_idx, 0]

    batch_obs[:, :, 1:] = y[idx, cond_idx, :].view(batch_size, len(cond_idx), dy)
    
    return batch_obs, y[idx, :, :]

In [8]:
test_ind = 0
tar = torch.linspace(0, 1, t_steps).unsqueeze(0).unsqueeze(-1).repeat(batch_size, 1, 1)

obs, real_y = get_batch(list(range(batch_size)), [test_ind])

print(obs.shape, real_y.shape, tar.shape)

with torch.no_grad():
    pred_wta, gate = wta(obs, tar)

out_traj = pred_wta[0, 0, :, :dy].cpu().numpy()

torch.Size([1, 1, 63]) torch.Size([1, 781, 62]) torch.Size([1, 781, 1])


In [9]:
print(out_traj.shape)

(781, 62)


In [10]:
from dm_control.locomotion.walkers.cmu_humanoid import _POSITION_ACTUATORS

#dm_control_dim_id: mocap_dim_id
indices = {0:16,1:17,2:18,3:50,4:51,5:52,6:47,7:48,8:49,9:19,
           10:20,11:21,12:53,13:54,14:55,15:0,16:1,17:2,18:28,19:29,
           20:38,21:39,22:40,23:41,24:46,25:36,26:37,27:33,28:42,29:43,
           30:3,31:4,32:13,33:14,34:15,35:22,36:27,37:11,38:12,39:8,
           40:23,41:24,42:30,43:31,44:32,45:44,46:34,47:35,48:45,49:5,
           50:6,51:7,52:25,53:9,54:10,55:26}

dm_traj = torch.zeros((t_steps, 56))
for i in range(t_steps):
    for j in range(56):
        dm_traj[i, j] = torch.tensor(out_traj[i, indices[j]])

In [2]:
from dm_control import composer
from dm_control.locomotion.arenas import floors
from dm_control.locomotion.mocap import cmu_mocap_data
from dm_control.locomotion.mocap import loader
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import utils
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.walkers import initializers
from dm_control.suite.wrappers import action_noise
import numpy as np
import tree
import mujoco

class StandInitializer(initializers.WalkerInitializer):
    def __init__(self):
        ref_path = cmu_mocap_data.get_path_for_cmu(version='2020')
        mocap_loader = loader.HDF5TrajectoryLoader(ref_path)
        trajectory = mocap_loader.get_trajectory('CMU_049_06')
        clip_reference_features = trajectory.as_dict()
        clip_reference_features = tracking._strip_reference_prefix(clip_reference_features, 'walker/')
        self._stand_features = tree.map_structure(lambda x: x[0], clip_reference_features)

    def initialize_pose(self, physics, walker, random_state):
        del random_state
        utils.set_walker_from_features(physics, walker, self._stand_features)
        mujoco.mj_kinematics(physics.model.ptr, physics.data.ptr)

In [7]:
import time
from dm_control import viewer

from dm_control import composer
from dm_control.locomotion import arenas

from dm_control.locomotion.mocap import cmu_mocap_data
from dm_control.locomotion.tasks import go_to_target

from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.suite import humanoid_CMU

initializer = StandInitializer()
walker = cmu_humanoid.CMUHumanoidPositionControlledV2020(initializer=initializer)

# Build an empty arena.
arena = arenas.Floor()

env = humanoid_CMU.stand()

# Viewer for visualization
viewer.launch(env)
te = env.reset()
# print(te[3])

# Run the simulation
for t in range(3):
    a = walker.cmu_pose_to_actuation(out_traj[t])
    s = env.step(a)
    # print(s)
    # print(s[3].shape)

# Close the environment
env.close()

NameError: name 'out_traj' is not defined

In [21]:
# for k in s[3].keys():
#     print(k, s[3][k].shape)

vals = s[3].values()
values_list = [v.flatten() for v in vals]
v = np.concatenate(values_list)
print(v.shape)

(227,)
